# Part 1

In [30]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import networkx as nx
from collections import Counter
import random
import time
import math

In [9]:
a_q=pd.read_csv('a2q',sep=' ', names=['u','v','t'])  #answers to questions
c_a=pd.read_csv('c2a',sep=' ', names=['u','v','t'])  #comments to answers
c_q=pd.read_csv('c2q',sep=' ', names=['u','v','t'])  #comments to questions

In [10]:
print(a_q.shape,c_a.shape,c_q.shape)

(17823525, 3) (25405374, 3) (20268151, 3)


In [11]:
#change time to hours
a_q['t'] = a_q['t'].div(3600)
c_a['t'] = c_a['t'].div(3600)
c_q['t'] = c_q['t'].div(3600)

#add type column to dataframes
a_q['type'] = 'a_q'
c_a['type'] = 'c_a'
c_q['type'] = 'c_q'

#add weights column
a_q['weight'] = 1
c_a['weight'] = 1
c_q['weight'] = 1

#sort by time
a_q=a_q.sort_values('t')
c_a=c_a.sort_values('t')
c_q=c_q.sort_values('t')

In [12]:
def time_maxmin(data,min,max): #Retrieve maximum and minimum timestamps
    m = data['t'].min()
    M = data['t'].max()
    if m < min:
        min = m 
    if M > max:
        max = M
    return min, max

In [13]:
def create_graph(data1,time_window,self_loops=0):
    #remove rows with u=v
    if self_loops == 0: 
        data1 = data1[data1['u'] != data1['v']]
    
    #remove rows not having time 't' in given time interval
    data1 = data1[data1['t'].between(time_window[0], time_window[1])]
    
    #assign weight --> 1 for every edge ,+1 every time u,v (interact)
    D = data1[['u','v']]
    D = D[D.duplicated(keep=False)]
    D = D.groupby(list(D)).apply(lambda x: tuple(x.index)).tolist()
    for d in D:
        ind = list(d)
        val = len(ind)
        drop = ind[:-1]
        data1.at[ind[-1], 'weight'] = val
        data1 = data1.drop(index = drop)
    
    G = nx.from_pandas_edgelist(data1, 'u', 'v', ['t', 'type', 'weight'], create_using = nx.MultiDiGraph())
    return G

In [14]:
m = 10**(10)
M = 0
m, M = time_maxmin(a_q, m, M)
m, M = time_maxmin(c_a, m, M)
m, M = time_maxmin(c_q, m, M)
print(m, M)

338213.2991666667 404798.1744444444


In [115]:
#obviously choose a time window that's inside (m,M)
time_window = (m, m+350)
A = create_graph(a_q, time_window)
B = create_graph(c_a, time_window)
C = create_graph(c_q, time_window)

In [116]:
#merge the graphs
G = nx.compose_all([A, B, C])

# Part 2

## function 1 

It takes in input:

    One of the 3 graphs

The output should return:

    Whether the graph is directed or not
    Number of users
    Number of answers/comments
    Average number of links per user
    Density degree of the graph
    Whether the graph is sparse or dense


In [117]:
def getFeatures(G):
    feats=[] 
    #m = G.number_of_edges()    = n_interactions
    #n = sum(G.nodes)           = n_users
    
    direct = False
    n_interactions = 0
    check_dir = {}
    users=[]
    for u,v,attr in (G.edges(data=True)):
        if ((u,v) or (v,u)) not in check_dir:
            check_dir[(u,v)]=0
        else:
            #print(u,v)
            direct=True
        
        if u not in users: users.append(u)
        if v not in users: users.append(v)

        n_interactions += 1
    
    n_users = len(users)
     
    avg_links = 0
    density = 0
    g = None
    if n_users>0: 
        avg_links = n_interactions/n_users
        
        if direct:
            density = 2*n_interactions/(n_users*(n_users-1))
        else:
            density = n_interactions/(n_users*(n_users-1))
                                
        if density>0.5:
            g='dense'
        else:
            g='sparse'
        
    feats.extend((direct, n_users, n_interactions, avg_links, density, g))
    
    return feats

## function 2 

It takes in input:

    A user/node
    An interval of time
    One of the following metrics: Betweeness 1, PageRank, ClosenessCentrality 3, DegreeCentrality

The output should return:

    The value of the given metric applied over the complete graph for the given interval of time

Give an explanaition regarding the features of the user based on all of the metrics (e.g. if the betweeness metric is high, what does this mean in practice, what if the betweeness is low but it has a high PageRank value, etc.)

In [118]:
#costruisco funzione per grafi in cui è gia stato preso in considerazione l'intervallo di tempo
def dist(G, u, metric, time_interval=100, alpha=0.2):  #recommended time_interval=100
    values = getFeatures(G)
    n = values[1]
    
    if metric == 'Betweeness':
        return Betweenness(G,u)
    
    elif metric == 'PageRank':
        return RandomSurfer(G, alpha, time_interval)
    
    elif metric == 'Closeness':
        return Closeness(G,u)
    
    elif metric == 'Degree':
        return (G.degree(int(u))/(n-1))
    
    else:
        raise('Metric not allowed')

### PageRank (Random surfer model)

In [119]:
def build_adj_matrix(G): #create adjacency matrix of the graph
    n = len(G)
    if n == 0: raise('No nodes found')
    else: #we opted for doing it witha  dictionary in which we store a 1 if there is a edge between the 2 nodes
        ones = {} 
        
        for u,v in sorted(list(G.edges())):
            ones[(u,v)] = 1 
    return ones

In [120]:
def mapper(G): #return dictionary with nodes as keys and index in matrix as value
    conv={}
    counter=0
    for u in sorted(G.nodes):
        conv[u]=counter
        counter+=1
    return conv

In [121]:
def no_tp_P(G,mat,mapper):
    keys = list(mat.keys())
    c = Counter(u for u,v in keys) #count number of occurances of each node, so we know by what value we have to divide to obtain P^{RW}
    for source in sorted(list(G)):
        for (u,v) in keys:
            if source == u and c[source] > 0:
                mat[(u,v)] = 1/c[source] #we divide by the value here
    keys = list(mat.keys())
    M=np.zeros(shape=(len(G),len(G))) #build ndarray (aka matrix of P^{RW}) nxn 
    for (u,v) in keys:
        value=mat[(u,v)]
        i=mapper[u]
        j=mapper[v]
        M[i][j]=value
    return M

In [122]:
def build_P(M, alpha, n): #create the P matrix from given alpha and P^{RW}
    P = (alpha/n * np.ones((n,n))) + ((1-alpha)*(M))   
    return P

In [123]:
def RandomSurfer(G, alpha, T=100): #T=time, aka n_iterations
    n = len(G)
    print('sit back and relax, this is going to take some time')
    #Build P
    mat = build_adj_matrix(G) 
    mapp = mapper(G)
    mat = no_tp_P(G,mat,mapp)
    P = build_P(mat, alpha, n)
    
    #randomize the starting point
    start = random.randint(0,n)
    #retrieve node given its index in amtrix
    key_list = list(mapp.keys())
    val_list = list(mapp.values())
    s = key_list[val_list.index(start)] #s=starting node
    
    print(('starting from node {}').format(s))
    
    #initialize vecotr with all zeros except 1 at starting point
    q0=np.zeros((n,1))
    q0[start]=1
    store_value=[]
    conv=False
    for t in tqdm(range(T)):
        q = np.matmul(q0.T, np.linalg.matrix_power(P, t)) #q_t=q0 * P^t
        if (t>0) and (np.array_equal(q, store_value[-1])):
            print(('converged in {} steps').format(t-1))
            q=store_value[-2] #else we are taking a all zeros array
            conv=True
            break
        store_value.append(q)
    if conv == False: print("didn't converge")
    return q

In [ ]:
RandomSurfer(G, 0.2, T=100)

### Closeness

In [124]:
def shortest_path(G,source,target):
    dist = {}
    prev = {}
    nodes_list=[]
    #Initializatio
    for node in nx.nodes(G):
        dist.update({node : math.inf}) # A dict with the nodes and the distances (We initialize all the distances as + infinite)
        prev.update({node : None}) # A dict with the previous node of each node 
        nodes_list.append(node) # A list with the nodes that have not been visited
    dist[source] = 0 # We initialize the distance of the source node as 0
    
    while len(nodes_list) != 0: # We will do the next steps until we'll have visited all the nodes are we'll have found the target node
       
        # the algorithm starts visiting the node with the lower distance
        dist_list =[] 
        for node in nodes_list:
            dist_list.append(dist[node])
        u = nodes_list[dist_list.index(min(dist_list))]
        nodes_list.remove(u) # removing the node from the list of the nodes that have not been visited
        
        #If the visited node il the target stop the visits
        if u == target:
            break
            
        #Otherwise we look at the neighbors
        for neighbor in G.neighbors(u):
            #computing the new distances for all neighbors
            new_dist = dist[u] + G[u][neighbor][0]['weight']
            #If the new distance is lower than the previous one we update di distances dict
            if dist[neighbor] > new_dist:
                dist[neighbor] = new_dist
                prev[neighbor] = u
                
    #If the target has not a previous node it means that there is nott a path (the graph is not connected) so the algorithm
    #returns 'not possible'
    if prev[target] == None:
        return ('Not possible', [-1])
    
    #creating a list with the path from the source node to the target
    path_node = target
    path = [path_node]
    
    while path_node != source:
    
        path.insert(0,prev[path_node])
        path_node = prev[path_node]
    
    #The algorithm returns the distance of the target and the path
    return dist[target], path


In [127]:
def Closeness(G,u):
    n=len(G)
    somma=0
    for v in tqdm(G.nodes):
        dist,path=shortest_path(G,u,v)
        if (dist!='Not possible') and (dist!=0):
            somma+=int(dist)
    #print(somma)
    if somma>0: return ((n-1)/somma)
    else: raise('Error: sum=0')

In [128]:
Closeness(G,13)

100%|█████████████████████████████████████████| 775/775 [00:18<00:00, 41.49it/s]


0.5108910891089109

### Betweenness

In [129]:
def Betweenness(G,q):
    print("You chose to use betweenness, we suggest you to go out and take a hangover, you'll be fine before this is over")
    n=len(G)
    somma_q = 0
    somma = 0
    for u in tqdm(G.nodes):
        for v in (G.nodes):
            dist,path = shortest_path(G,u,v)
            if (dist!='Not possible') and (dist!=0):
                somma+=int(dist)
                if q in path:
                    somma_q+=int(dist)
    
    num = 2*somma_q/somma_v
    den = n*n - 3*n + 2
    if den>0: return num/den
    else: raise('Error: denominator=0')

In [ ]:
Betweenness(G,13)

You chose to use betweenness, we suggest you to go out and take a hangover, you'll be fine before this is over


 21%|████████                               | 159/775 [36:51<2:51:57, 16.75s/it]